<a href="https://colab.research.google.com/gist/sajedeh2112/8aac305aa16006f347dc17b6a3394552/flood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare data

In [ ]:
!wget "https://jarkani.ir/flood/Australia_flood_9_10_11.txt"

--2022-04-30 09:37:32--  https://jarkani.ir/flood/Australia_flood_9_10_11.txt
Resolving jarkani.ir (jarkani.ir)... 116.202.89.179
Connecting to jarkani.ir (jarkani.ir)|116.202.89.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 309348 (302K) [text/plain]
Saving to: ‘Australia_flood_9_10_11.txt’

Australia_flood_9_1 100%[===================>] 302.10K  1014KB/s    in 0.3s    

2022-04-30 09:37:34 (1014 KB/s) - ‘Australia_flood_9_10_11.txt’ saved [309348/309348]



In [ ]:
import numpy as np
from keras.utils import np_utils

In [ ]:
file = open("Australia_flood_9_10_11.txt", "r", encoding='Latin-1')
content = file.read()
lines = content.split("\n")

In [ ]:
# from random import shuffle

# shuffle(lines)
# content = "\n".join(lines)
# file = open("Australia_flood_9_10_11.txt", "w", encoding='Latin-1')
# file.write(content)
# file.close()

In [ ]:
# Create empty array for this groups:
# No need (0)
# Need    (1)
groups = np.zeros((len(lines), 1))

# Create empty matrix for
# injured
# live
# medical
# transmit
what_needed = np.zeros((len(lines), 4))

texts = []

In [ ]:
for i in range(len(lines)):
  parts = lines[i].split("\t")
  group, *extra = parts[0].split(" ")[0]

  groups[i] = float(group)

  if (group == '1'):
    what_needed[i, 0] = '1' in extra  # injured
    what_needed[i, 1] = '2' in extra  # live
    what_needed[i, 2] = '3' in extra  # medical
    what_needed[i, 3] = '4' in extra  # transmit

  texts.append(parts[-1])

In [ ]:
# groups

In [ ]:
# what_needed[:100]

In [ ]:
y1 = groups
y2 = what_needed
texts = np.array(texts)

# print(y1.shape)
# print(y2.shape)

In [ ]:
train_test_split = 1000

y1_train = y1[:train_test_split]
y1_test = y1[train_test_split:]

train_need_help_indexes = np.where(y1_train == 1.)[0]
test_need_help_indexes = np.where(y1_test == 1.)[0]

y2_train = y2[:train_test_split][train_need_help_indexes]
y2_test = y2[train_test_split:][test_need_help_indexes]

train_texts = texts[:train_test_split]
test_texts = texts[train_test_split:]

In [ ]:
y1_train.mean(), y1_test.mean()

(0.258, 0.23655913978494625)

In [ ]:
y2_train.mean(axis=0), y2_test.mean(axis=0)

(array([0.16666667, 0.29069767, 0.01937984, 0.58914729]),
 array([0.1       , 0.35454545, 0.01818182, 0.58181818]))

In [ ]:
print("############### Shape ###################")
print("Train Y1 shape: ", y1_train.shape)
print("Test Y1 shape:  ", y1_test.shape)

print("Train Y2 shape: ", y2_train.shape)
print("Test Y2 shape:  ", y2_test.shape)

print("Train samples: ", len(train_texts))
print("Test samples:  ", len(test_texts))
print("#########################################")

############### Shape ###################
Train Y1 shape:  (1000, 1)
Test Y1 shape:   (465, 1)
Train Y2 shape:  (258, 4)
Test Y2 shape:   (110, 4)
Train samples:  1000
Test samples:   465
#########################################


In [ ]:
class Vocabulary:
    def __init__(self):
      self.word2index = {"":0}
      self.word2count = {"":0}
      self.index2word = [""]
      self.num_words = 1

      escape_chars = {}
      for char in list("!.#,?@$%&*()+-[]\/|:"):
        escape_chars[char] = ''
      self.translate = str.maketrans(escape_chars)

    def word_correction(self, word):
      return word.lower().strip().translate(self.translate)

    def add_word(self, word):
      if (word[:8] == 'https://'):
        return

      word = self.word_correction(word)
      if len(word) == 0:
        return

      if word not in self.word2index:
          # First entry of word into vocabulary
          self.word2index[word] = self.num_words
          self.word2count[word] = 1
          self.index2word.append(word)
          self.num_words += 1
      else:
          # Word exists; increase word count
          self.word2count[word] += 1

    def load(self, texts):
      for text in texts:
        for word in text.split(' '):
          self.add_word(word)

    def has(self, word):
      return word in self.word2index

    def to_word(self, index):
        return self.index2word[index]

    def to_index(self, word):
        return self.word2index[word]

    def histogram(self):
      counts = np.array(list(voc.word2count.values()))
      return np.histogram(counts, bins=counts.max(), range=(0, counts.max()))

    def encode(self, text):
      encode = []
      for word in text.split(' '):
        word = self.word_correction(word)
        if len(word) > 0 and self.has(word):
          encode.append(self.word2index[word])

      return encode

    def decode(self, encoded):
      decoded = []
      for i in encoded:
        if i < 1:
          continue
        decoded.append(self.index2word[int(i)])

      return " ".join(decoded)

    def threshold(self, threshold):
      w2i = {}
      i2w = [""]
      num = 1
      c = {}

      for word in self.index2word:
        if self.word2count[word] >= threshold:
          w2i[word] = num
          i2w.append(word)
          c[word] = self.word2count[word]
          num += 1

      self.word2count = c
      self.word2index = w2i
      self.index2word = i2w
      self.num_words = num

In [ ]:
voc = Vocabulary()
voc.load(texts)

# See word repeat
# print(voc.histogram()[0][:100])

# Remove words that repeated less thar 4
voc.threshold(4)

# See all words in dictionary
# print("Words: ", voc.index2word)

In [ ]:
#############################################
# We can encode and decode a text like this:
# sample = texts[10]
# encoded = voc.encode(sample)
# decoded = voc.decode(encoded)

# print("Original text: ", sample)
# print("Encoded:       ", encoded)
# print("Decoded:       ", decoded)
##############################################

In [ ]:
# Encode entire texts
encoded_list = []
for text in texts:
  encoded_list.append(voc.encode(text))

In [ ]:
# Find appropriate sequence length:
# counts = np.array(list(map(lambda l: len(l), encoded_list[:train_test_split])))
# np.histogram(counts, bins=counts.max(), range=(0, counts.max()))
# Seems 42 is fine for sequence length

In [ ]:
MAX_LENGTH = 42
x = np.zeros((len(encoded_list), MAX_LENGTH))

In [ ]:
for i in range(len(encoded_list)):
  encoded = encoded_list[i][:MAX_LENGTH]
  x[i, MAX_LENGTH-len(encoded):] = np.array(encoded)

In [ ]:
x1_train = x[:train_test_split]
x1_test = x[train_test_split:]

x2_train = x1_train[train_need_help_indexes]
x2_test = x1_test[test_need_help_indexes]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, BatchNormalization, Dense, Dropout, SimpleRNN, LSTM, GRU, Conv1D, MaxPooling1D, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras import Model
from sklearn.metrics import confusion_matrix



def early_stopping():
  return EarlyStopping(monitor="val_loss",
                         patience=3,
                         restore_best_weights=True,
                         verbose = 1)

# Model 1

## LSTM

In [ ]:
############################# Create models ##############################

# First create a model to classify need help or not
model = Sequential(name="model1_LSTM")
model.add(Embedding(voc.num_words, 128, input_length=MAX_LENGTH))
model.add(LSTM(64, return_sequences=False))
model.add(BatchNormalization())
model.add(Dense(32, activation='tanh'))
model.add(Dense(16, activation='tanh'))
model.add(Dense(8, activation='tanh'))
model.add(Dropout(0.75))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

model.summary()

Model: "model1_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 42, 128)           84224     
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                       

In [ ]:
class_weight = {0: 1., 1: 4.}

In [ ]:
model.fit(x1_train, y1_train, batch_size=128, epochs=100,class_weight = class_weight , callbacks=[early_stopping()], validation_data=(x1_test, y1_test))

Epoch 1/100
8/8 [==============================] - 1s 112ms/step - loss: 0.3658 - accuracy: 0.8950 - val_loss: 0.2710 - val_accuracy: 0.9161
Epoch 2/100
8/8 [==============================] - 1s 100ms/step - loss: 0.3153 - accuracy: 0.9000 - val_loss: 0.2526 - val_accuracy: 0.9527
Epoch 3/100
8/8 [==============================] - 1s 101ms/step - loss: 0.2987 - accuracy: 0.9070 - val_loss: 0.2419 - val_accuracy: 0.9613
Epoch 4/100
8/8 [==============================] - 1s 99ms/step - loss: 0.3262 - accuracy: 0.9020 - val_loss: 0.2670 - val_accuracy: 0.9183
Epoch 5/100
8/8 [==============================] - 1s 104ms/step - loss: 0.3222 - accuracy: 0.9280 - val_loss: 0.3226 - val_accuracy: 0.8968
Epoch 6/100
8/8 [==============================] - 1s 105ms/step - loss: 0.2997 - accuracy: 0.9020 - val_loss: 0.3270 - val_accuracy: 0.9011
Epoch 6: early stopping


In [ ]:
# model.save("model1_LSTM.h5")

In [ ]:
# model = load_model("model1_LSTM.h5")

In [ ]:
model.evaluate(x1_test, y1_test)

15/15 [==============================] - 0s 11ms/step - loss: 0.2419 - accuracy: 0.9613


[0.24191151559352875, 0.9612902998924255]

In [ ]:
predict = model.predict(x1_test)
y_pred = predict.round()
y_true = y1_test

In [ ]:
matrix = confusion_matrix(y_true, y_pred)
print("Confusion matrix:\n", matrix)

Confusion matrix:
 [[347   8]
 [ 10 100]]


## RNN

In [ ]:
############################# Create models ##############################

# First create a model to classify need help or not
model = Sequential(name="model1_RNN")
model.add(Embedding(voc.num_words, 128, input_length=MAX_LENGTH))
model.add(SimpleRNN(64, return_sequences=False))
model.add(BatchNormalization())
model.add(Dense(32, activation='tanh'))
model.add(Dense(16, activation='tanh'))
model.add(Dense(8, activation='tanh'))
model.add(Dropout(0.75))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

model.summary()

Model: "model1_RNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 42, 128)           84224     
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                12352     
                                                                 
 batch_normalization_2 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 16)                528       
                                                                 
 dense_6 (Dense)             (None, 8)                 136       
                                                        

In [ ]:
model.fit(x1_train, y1_train, batch_size=128, epochs=100, class_weight=class_weight, callbacks=[early_stopping()], validation_data=(x1_test, y1_test))

Epoch 1/100
8/8 [==============================] - 1s 40ms/step - loss: 0.6455 - accuracy: 0.8070 - val_loss: 0.3198 - val_accuracy: 0.9118
Epoch 2/100
8/8 [==============================] - 0s 36ms/step - loss: 0.6216 - accuracy: 0.8390 - val_loss: 0.3091 - val_accuracy: 0.9505
Epoch 3/100
8/8 [==============================] - 0s 38ms/step - loss: 0.6081 - accuracy: 0.8290 - val_loss: 0.3000 - val_accuracy: 0.9011
Epoch 4/100
8/8 [==============================] - 0s 35ms/step - loss: 0.5688 - accuracy: 0.8430 - val_loss: 0.2961 - val_accuracy: 0.8989
Epoch 5/100
8/8 [==============================] - 0s 36ms/step - loss: 0.5745 - accuracy: 0.8220 - val_loss: 0.2829 - val_accuracy: 0.9161
Epoch 6/100
8/8 [==============================] - 0s 37ms/step - loss: 0.5662 - accuracy: 0.8180 - val_loss: 0.2568 - val_accuracy: 0.9290
Epoch 7/100
8/8 [==============================] - 0s 36ms/step - loss: 0.5658 - accuracy: 0.8410 - val_loss: 0.2527 - val_accuracy: 0.9290
Epoch 8/100
8/8 [===

In [ ]:
# model.save("model1_RNN.h5")

In [ ]:
# model = load_model("model1_RNN.h5")

In [ ]:
model.evaluate(x1_test, y1_test)

15/15 [==============================] - 0s 5ms/step - loss: 0.2259 - accuracy: 0.9527


[0.22593475878238678, 0.9526881575584412]

In [ ]:
predict = model.predict(x1_test)
y_pred = predict.round()
y_true = y1_test

In [ ]:
matrix = confusion_matrix(y_true, y_pred)
print("Confusion matrix:\n", matrix)

Confusion matrix:
 [[350   5]
 [ 17  93]]


## GRU

In [ ]:
############################# Create models ##############################

# First create a model to classify need help or not
model = Sequential(name="model1_GRU")
model.add(Embedding(voc.num_words, 128, input_length=MAX_LENGTH))
model.add(GRU(64, return_sequences=False))
model.add(BatchNormalization())
model.add(Dense(32, activation='tanh'))
model.add(Dense(16, activation='tanh'))
model.add(Dense(8, activation='tanh'))
model.add(Dropout(0.75))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

model.summary()

Model: "model1_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 42, 128)           84224     
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 batch_normalization_4 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 dense_8 (Dense)             (None, 32)                2080      
                                                                 
 dense_9 (Dense)             (None, 16)                528       
                                                                 
 dense_10 (Dense)            (None, 8)                 136       
                                                        

In [ ]:
model.fit(x1_train, y1_train, batch_size=128, epochs=100, class_weight=class_weight, callbacks=[early_stopping()], validation_data=(x1_test, y1_test))

Epoch 1/100
8/8 [==============================] - 2s 89ms/step - loss: 0.6016 - accuracy: 0.8800 - val_loss: 0.5466 - val_accuracy: 0.9290
Epoch 2/100
8/8 [==============================] - 1s 86ms/step - loss: 0.5880 - accuracy: 0.8680 - val_loss: 0.5453 - val_accuracy: 0.8753
Epoch 3/100
8/8 [==============================] - 1s 84ms/step - loss: 0.5591 - accuracy: 0.8610 - val_loss: 0.5681 - val_accuracy: 0.8473
Epoch 4/100
8/8 [==============================] - 1s 84ms/step - loss: 0.5091 - accuracy: 0.8730 - val_loss: 0.7506 - val_accuracy: 0.3183
Epoch 5/100
8/8 [==============================] - 1s 86ms/step - loss: 0.5058 - accuracy: 0.8760 - val_loss: 0.6467 - val_accuracy: 0.4817
Epoch 5: early stopping


In [ ]:
# model.save("model1_GRU.h5")

In [ ]:
# model = load_model("model1_GRU.h5")

In [ ]:
model.evaluate(x1_test, y1_test)

15/15 [==============================] - 0s 9ms/step - loss: 0.5453 - accuracy: 0.8753


[0.5452933311462402, 0.875268816947937]

In [ ]:
predict = model.predict(x1_test)
y_pred = predict.round()
y_true = y1_test

In [ ]:
matrix = confusion_matrix(y_true, y_pred)
print("Confusion matrix:\n", matrix)

Confusion matrix:
 [[307  48]
 [ 10 100]]


## CNN

In [ ]:
############################# Create models ##############################

# First create a model to classify need help or not
model = Sequential(name="model1_CNN")
model.add(Embedding(voc.num_words, 128, input_length=MAX_LENGTH))
model.add(Conv1D(64, kernel_size=3))
model.add(MaxPooling1D())
model.add(Conv1D(64, kernel_size=3))
model.add(MaxPooling1D())
model.add(Conv1D(128, kernel_size=3))
model.add(MaxPooling1D())
model.add(Conv1D(128, kernel_size=3))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(32, activation='tanh'))
model.add(Dense(16, activation='tanh'))
model.add(Dense(8, activation='tanh'))
model.add(Dropout(0.75))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

model.summary()

Model: "model1_CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 42, 128)           84224     
                                                                 
 conv1d_8 (Conv1D)           (None, 40, 64)            24640     
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 20, 64)           0         
 1D)                                                             
                                                                 
 conv1d_9 (Conv1D)           (None, 18, 64)            12352     
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 9, 64)            0         
 1D)                                                             
                                                                 
 conv1d_10 (Conv1D)          (None, 7, 128)            2

In [ ]:
model.fit(x1_train, y1_train, batch_size=128, epochs=100, class_weight=class_weight, callbacks=[early_stopping()], validation_data=(x1_test, y1_test))

Epoch 1/100
8/8 [==============================] - 1s 57ms/step - loss: 0.5087 - accuracy: 0.8820 - val_loss: 0.4341 - val_accuracy: 0.9548
Epoch 2/100
8/8 [==============================] - 0s 50ms/step - loss: 0.5147 - accuracy: 0.8730 - val_loss: 0.4370 - val_accuracy: 0.9548
Epoch 3/100
8/8 [==============================] - 0s 52ms/step - loss: 0.5125 - accuracy: 0.8700 - val_loss: 0.4026 - val_accuracy: 0.9527
Epoch 4/100
8/8 [==============================] - 0s 53ms/step - loss: 0.4808 - accuracy: 0.8760 - val_loss: 0.4036 - val_accuracy: 0.9527
Epoch 5/100
8/8 [==============================] - 0s 51ms/step - loss: 0.4464 - accuracy: 0.8790 - val_loss: 0.4279 - val_accuracy: 0.9376
Epoch 6/100
8/8 [==============================] - 0s 54ms/step - loss: 0.4310 - accuracy: 0.8870 - val_loss: 0.4353 - val_accuracy: 0.9355
Epoch 6: early stopping


In [ ]:
# model.save("model1_CNN.h5")

In [ ]:
# model = load_model("model1_CNN.h5")

In [ ]:
model.evaluate(x1_test, y1_test)

15/15 [==============================] - 0s 5ms/step - loss: 0.4026 - accuracy: 0.9527


[0.4025914967060089, 0.9526881575584412]

In [ ]:
predict = model.predict(x1_test)
y_pred = predict.round()
y_true = y1_test

In [ ]:
matrix = confusion_matrix(y_true, y_pred)
print("Confusion matrix:\n", matrix)

Confusion matrix:
 [[344  11]
 [ 11  99]]


## MLP

In [ ]:
############################# Create models ##############################

# First create a model to classify need help or not
model = Sequential(name="model1_MLP")
model.add(Embedding(voc.num_words, 128, input_length=MAX_LENGTH))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(32, activation='tanh'))
model.add(Dense(16, activation='tanh'))
model.add(Dense(8, activation='tanh'))
model.add(Dropout(0.75))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

model.summary()

Model: "model1_MLP"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 42, 128)           84224     
                                                                 
 flatten_3 (Flatten)         (None, 5376)              0         
                                                                 
 batch_normalization_12 (Bat  (None, 5376)             21504     
 chNormalization)                                                
                                                                 
 dense_24 (Dense)            (None, 32)                172064    
                                                                 
 dense_25 (Dense)            (None, 16)                528       
                                                                 
 dense_26 (Dense)            (None, 8)                 136       
                                                        

In [ ]:
model.fit(x1_train, y1_train, batch_size=128, epochs=100, class_weight=class_weight, callbacks=[early_stopping()], validation_data=(x1_test, y1_test))

Epoch 1/100
8/8 [==============================] - 3s 48ms/step - loss: 1.3522 - accuracy: 0.5430 - val_loss: 0.4826 - val_accuracy: 0.7978
Epoch 2/100
8/8 [==============================] - 0s 23ms/step - loss: 1.0264 - accuracy: 0.6360 - val_loss: 0.4625 - val_accuracy: 0.7978
Epoch 3/100
8/8 [==============================] - 0s 24ms/step - loss: 0.9179 - accuracy: 0.6800 - val_loss: 0.4484 - val_accuracy: 0.7978
Epoch 4/100
8/8 [==============================] - 0s 24ms/step - loss: 0.8142 - accuracy: 0.7170 - val_loss: 0.4349 - val_accuracy: 0.8172
Epoch 5/100
8/8 [==============================] - 0s 24ms/step - loss: 0.7502 - accuracy: 0.7440 - val_loss: 0.4236 - val_accuracy: 0.8172
Epoch 6/100
8/8 [==============================] - 0s 24ms/step - loss: 0.7241 - accuracy: 0.7530 - val_loss: 0.4154 - val_accuracy: 0.8172
Epoch 7/100
8/8 [==============================] - 0s 23ms/step - loss: 0.6757 - accuracy: 0.7760 - val_loss: 0.4123 - val_accuracy: 0.8172
Epoch 8/100
8/8 [===

In [ ]:
# model.save("model1_MLP.h5")

In [ ]:
# model = load_model("model1_MLP.h5")

In [ ]:
model.evaluate(x1_test, y1_test)

15/15 [==============================] - 0s 3ms/step - loss: 0.1816 - accuracy: 0.9570


[0.1816110610961914, 0.9569892287254333]

In [ ]:
predict = model.predict(x1_test)
y_pred = predict.round()
y_true = y1_test

In [ ]:
matrix = confusion_matrix(y_true, y_pred)
print("Confusion matrix:\n", matrix)

Confusion matrix:
 [[352   3]
 [ 17  93]]


## Bert

In [ ]:
!pip install -q -U "tensorflow-text==2.8.*"

     |████████████████████████████████| 4.9 MB 5.6 MB/s 
     |████████████████████████████████| 462 kB 72.7 MB/s 


In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text  # A dependency of the preprocessing model

bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4",trainable=False)

In [ ]:
# model = Sequential()
# model.add(Dense(64))
# model.add(Dense(32))
# model.add(Dense(1))


# input = Input()
# layer1 = Dense(64)(input)
# layer2 = Dense(32)(layer1)
# output = Dense(1)(layer2)

# model = Model(inputs=input, outputs=output)

In [ ]:
import tensorflow as tf


text_input = Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)


y = LSTM(64)(outputs['sequence_output'])
y = Dense(16, activation='tanh')(y)
y = Dense(8, activation='tanh')(y)
y = Dense(1, activation='sigmoid')(y)


model = Model(inputs=[text_input], outputs = [y], name="model1_Bert")

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

model.summary()

Model: "model1_Bert"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                

In [ ]:
model.fit(train_texts,y1_train, batch_size=64, epochs=100, class_weight=class_weight, callbacks=[early_stopping()], validation_data=(test_texts, y1_test))

Epoch 1/100
16/16 [==============================] - 647s 41s/step - loss: 1.0354 - accuracy: 0.5830 - val_loss: 0.4736 - val_accuracy: 0.7978
Epoch 2/100
16/16 [==============================] - 621s 40s/step - loss: 0.6925 - accuracy: 0.8080 - val_loss: 0.6583 - val_accuracy: 0.6215
Epoch 3/100
16/16 [==============================] - 621s 40s/step - loss: 0.5595 - accuracy: 0.8230 - val_loss: 0.2854 - val_accuracy: 0.8925
Epoch 4/100
16/16 [==============================] - 609s 39s/step - loss: 0.5970 - accuracy: 0.8390 - val_loss: 0.3358 - val_accuracy: 0.8581
Epoch 5/100
16/16 [==============================] - 609s 39s/step - loss: 0.4155 - accuracy: 0.8810 - val_loss: 0.2604 - val_accuracy: 0.9097
Epoch 6/100
16/16 [==============================] - ETA: 0s - loss: 0.3555 - accuracy: 0.9180 

In [ ]:
# model.save("model1_Bert.h5")

In [ ]:
# model = load_model("model1_Bert.h5")

In [ ]:
model.evaluate(test_texts, y1_test)

In [ ]:
predict = model.predict(test_texts)
y_pred = predict.round()
y_true = y1_test

In [ ]:
matrix = confusion_matrix(y_true, y_pred)
print("Confusion matrix:\n", matrix)

# Model 2

## LSTM

In [ ]:
# Second model must define type of helps that needed
model2 = Sequential(name="model2_LSTM")
model2.add(Embedding(voc.num_words, 128, input_length=MAX_LENGTH))
model2.add(LSTM(64, return_sequences=False))
model2.add(BatchNormalization())
model2.add(Dense(32, activation='tanh'))
model2.add(Dense(16, activation='tanh'))
model2.add(Dropout(0.75))
model2.add(BatchNormalization())
model2.add(Dense(4, activation='sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

model2.summary()

Model: "model2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 42, 128)           84224     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 16)                528       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                       

In [ ]:
model2.fit(x2_train, y2_train, batch_size=64, epochs=100, callbacks=[early_stopping()], validation_data=(x2_test, y2_test))

Epoch 1/100
5/5 [==============================] - 4s 194ms/step - loss: 0.7756 - accuracy: 0.3837 - val_loss: 0.6745 - val_accuracy: 0.7545
Epoch 2/100
5/5 [==============================] - 0s 49ms/step - loss: 0.6326 - accuracy: 0.5814 - val_loss: 0.6625 - val_accuracy: 0.7909
Epoch 3/100
5/5 [==============================] - 0s 51ms/step - loss: 0.6037 - accuracy: 0.6977 - val_loss: 0.6522 - val_accuracy: 0.7727
Epoch 4/100
5/5 [==============================] - 0s 49ms/step - loss: 0.5692 - accuracy: 0.6860 - val_loss: 0.6485 - val_accuracy: 0.8091
Epoch 5/100
5/5 [==============================] - 0s 50ms/step - loss: 0.5558 - accuracy: 0.7016 - val_loss: 0.6433 - val_accuracy: 0.7818
Epoch 6/100
5/5 [==============================] - 0s 48ms/step - loss: 0.5294 - accuracy: 0.7597 - val_loss: 0.6400 - val_accuracy: 0.7909
Epoch 7/100
5/5 [==============================] - 0s 53ms/step - loss: 0.5165 - accuracy: 0.7481 - val_loss: 0.6343 - val_accuracy: 0.7909
Epoch 8/100
5/5 [==

In [ ]:
# model2.save("model2_LSTM.h5")

In [ ]:
# model2 = load_model("model2_LSTM.h5")

In [ ]:
model2.evaluate(x2_test, y2_test)

4/4 [==============================] - 1s 9ms/step - loss: 0.5658 - accuracy: 0.5364


[0.5658222436904907, 0.5363636612892151]

In [ ]:
predict = model2.predict(x2_test).round()
y_true = y2_test

## RNN

In [ ]:
# Second model must define type of helps that needed

model2 = Sequential(name="model2_RNN")
model2.add(Embedding(voc.num_words, 128, input_length=MAX_LENGTH))
model2.add(SimpleRNN(64, return_sequences=False))
model2.add(BatchNormalization())
model2.add(Dense(32, activation='tanh'))
model2.add(Dense(16, activation='tanh'))
model2.add(Dense(8, activation='tanh'))
model2.add(Dropout(0.75))
model2.add(BatchNormalization())
model2.add(Dense(4, activation='sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

model2.summary()

In [ ]:
model2.fit(x2_train, y2_train, batch_size=64, epochs=100, callbacks=[early_stopping()], validation_data=(x2_test, y2_test))

In [ ]:
# model2.save("model2_RNN.h5")

In [ ]:
# model2 = load_model("model2_LSTM.h5")

In [ ]:
model2.evaluate(x2_test, y2_test)

In [ ]:
predict = model2.predict(x2_test).round()
y_true = y2_test

## GRU

In [ ]:
# Second model must define type of helps that needed

model2 = Sequential(name="model2_GRU")
model2.add(Embedding(voc.num_words, 128, input_length=MAX_LENGTH))
model2.add(GRU(64, return_sequences=False))
model2.add(BatchNormalization())
model2.add(Dense(32, activation='tanh'))
model2.add(Dense(16, activation='tanh'))
model2.add(Dense(8, activation='tanh'))
model2.add(Dropout(0.75))
model2.add(BatchNormalization())
model2.add(Dense(4, activation='sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

model2.summary()

In [ ]:
# model2.save("model2_GRU.h5")

In [ ]:
# model2 = load_model("model2_GRU.h5")

In [ ]:
model2.evaluate(x2_test, y2_test)

In [ ]:
predict = model2.predict(x2_test).round()
y_true = y2_test

## CNN

In [ ]:
# Second model must define type of helps that needed

model2 = Sequential(name="model2_CNN")
model2.add(Embedding(voc.num_words, 128, input_length=MAX_LENGTH))
model2.add(Conv1D(64, kernel_size=3))
model2.add(MaxPooling1D())
model2.add(Conv1D(64, kernel_size=3))
model2.add(MaxPooling1D())
model2.add(Conv1D(128, kernel_size=3))
model2.add(MaxPooling1D())
model2.add(Conv1D(128, kernel_size=3))
model2.add(Flatten())
model2.add(BatchNormalization())
model2.add(Dense(32, activation='tanh'))
model2.add(Dense(16, activation='tanh'))
model2.add(Dense(8, activation='tanh'))
model2.add(Dropout(0.75))
model2.add(BatchNormalization())
model2.add(Dense(4, activation='sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

model2.summary()

In [ ]:
# model2.save("model2_CNN.h5")

In [ ]:
# model2 = load_model("model2_CNN.h5")

In [ ]:
model2.evaluate(x2_test, y2_test)

In [ ]:
predict = model2.predict(x2_test).round()
y_true = y2_test

## MLP

In [ ]:
# Second model must define type of helps that needed

model2 = Sequential(name="model1_GRU")
model2.add(Embedding(voc.num_words, 128, input_length=MAX_LENGTH))
model2.add(Flatten())
model2.add(BatchNormalization())
model2.add(Dense(32, activation='tanh'))
model2.add(Dense(16, activation='tanh'))
model2.add(Dense(8, activation='tanh'))
model2.add(Dropout(0.75))
model2.add(BatchNormalization())
model2.add(Dense(4, activation='sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

model2.summary()

In [ ]:
# model2.save("model2_MLP.h5")

In [ ]:
# model2 = load_model("model2_MLP.h5")

In [ ]:
model2.evaluate(x2_test, y2_test)

In [ ]:
predict = model2.predict(x2_test).round()
y_true = y2_test

## Bert

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4",trainable=False)

In [ ]:
text_input = Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)


y = LSTM(64)(outputs['sequence_output'])
y = Dense(16, activation='tanh')(y)
y = Dense(8, activation='tanh')(y)
y = Dense(4, activation='sigmoid')(y)


model = Model(inputs=[text_input], outputs = [y], name="model1_Bert")

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(train_texts[train_need_help_indexes],y2_train, batch_size=64, epochs=100, callbacks=[early_stopping()], validation_data=(test_texts[test_need_help_indexes], y2_test))

In [ ]:
# model2.save("model2_Bert.h5")

In [ ]:
# model2 = load_model("model2_Bert.h5")

In [ ]:
model2.evaluate(test_texts[test_need_help_indexes], y2_test)

In [ ]:
predict = model2.predict(x2_test).round()
y_true = y2_test

In [ ]:
print("All: ", len(y_true))
for i in range(4):
  print("#################################")
  print("Actual in class #"+str(i)+" : ", y_true[:, i].sum())
  print("Predicted in class #"+str(i)+" : ", predict[:, i].sum())

All:  110
#################################
Actual in class #0 :  11.0
Predicted in class #0 :  1.0
#################################
Actual in class #1 :  39.0
Predicted in class #1 :  17.0
#################################
Actual in class #2 :  2.0
Predicted in class #2 :  0.0
#################################
Actual in class #3 :  64.0
Predicted in class #3 :  106.0


In [ ]:
print("Total: ", len(y1_test))
print("Class #0", len(np.where(y1_test==0.0)[0]))
print("Class #1", len(np.where(y1_test==1.0)[0]))

Total:  465
Class #0 355
Class #1 110


In [ ]:
print("Only class #1")
print("Total: ", len(y2_test))
print("Class #11 \t", len(np.nonzero(np.abs(y2_test - np.array([1., 0., 0., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #12 \t", len(np.nonzero(np.abs(y2_test - np.array([0., 1., 0., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #13 \t", len(np.nonzero(np.abs(y2_test - np.array([0., 0., 1., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #14 \t", len(np.nonzero(np.abs(y2_test - np.array([0., 0., 0., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #112 \t", len(np.nonzero(np.abs(y2_test - np.array([1., 1., 0., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #113 \t", len(np.nonzero(np.abs(y2_test - np.array([1., 0., 1., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #114 \t", len(np.nonzero(np.abs(y2_test - np.array([1., 0., 0., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #123 \t", len(np.nonzero(np.abs(y2_test - np.array([0., 1., 1., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #124 \t", len(np.nonzero(np.abs(y2_test - np.array([0., 1., 0., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #134 \t", len(np.nonzero(np.abs(y2_test - np.array([0., 0., 1., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #1123 \t", len(np.nonzero(np.abs(y2_test - np.array([1., 1., 1., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #1124 \t", len(np.nonzero(np.abs(y2_test - np.array([1., 1., 0., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #1134 \t", len(np.nonzero(np.abs(y2_test - np.array([1., 0., 1., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #1234 \t", len(np.nonzero(np.abs(y2_test - np.array([0., 1., 1., 1.])).sum(axis=1) < 0.1)[0]))

Only class #1
Total:  110
Class #11 	 8
Class #12 	 38
Class #13 	 0
Class #14 	 58
Class #112 	 0
Class #113 	 0
Class #114 	 3
Class #123 	 0
Class #124 	 1
Class #134 	 2
Class #1123 	 0
Class #1124 	 0
Class #1134 	 0
Class #1234 	 0


In [ ]:
print("Prideicted Only class #1")
print("Class #11 \t", len(np.nonzero(np.abs(predict - np.array([1., 0., 0., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #12 \t", len(np.nonzero(np.abs(predict - np.array([0., 1., 0., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #13 \t", len(np.nonzero(np.abs(predict - np.array([0., 0., 1., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #14 \t", len(np.nonzero(np.abs(predict - np.array([0., 0., 0., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #112 \t", len(np.nonzero(np.abs(predict - np.array([1., 1., 0., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #113 \t", len(np.nonzero(np.abs(predict - np.array([1., 0., 1., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #114 \t", len(np.nonzero(np.abs(predict - np.array([1., 0., 0., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #123 \t", len(np.nonzero(np.abs(predict - np.array([0., 1., 1., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #124 \t", len(np.nonzero(np.abs(predict - np.array([0., 1., 0., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #134 \t", len(np.nonzero(np.abs(predict - np.array([0., 0., 1., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #1123 \t", len(np.nonzero(np.abs(predict - np.array([1., 1., 1., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #1124 \t", len(np.nonzero(np.abs(predict - np.array([1., 1., 0., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #1134 \t", len(np.nonzero(np.abs(predict - np.array([1., 0., 1., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #1234 \t", len(np.nonzero(np.abs(predict - np.array([0., 1., 1., 1.])).sum(axis=1) < 0.1)[0]))

Prideicted Only class #1
Class #11 	 1
Class #12 	 0
Class #13 	 0
Class #14 	 89
Class #112 	 0
Class #113 	 0
Class #114 	 0
Class #123 	 0
Class #124 	 17
Class #134 	 0
Class #1123 	 0
Class #1124 	 0
Class #1134 	 0
Class #1234 	 0


In [ ]:
# Predict with model2 acording to model1
model1_predict_need = np.nonzero(model.predict(x1_test).round() == 1.)[0]

x3_test = x[train_test_split:][model1_predict_need]
y3_test = y2[train_test_split:][model1_predict_need]

In [ ]:
print(len(y3_test))

62


In [ ]:
predict = model2.predict(x3_test).round()
y_true = y2_test

In [ ]:
print("Prideicted Only class #1")
print("Class #11 \t", len(np.nonzero(np.abs(predict - np.array([1., 0., 0., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #12 \t", len(np.nonzero(np.abs(predict - np.array([0., 1., 0., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #13 \t", len(np.nonzero(np.abs(predict - np.array([0., 0., 1., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #14 \t", len(np.nonzero(np.abs(predict - np.array([0., 0., 0., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #112 \t", len(np.nonzero(np.abs(predict - np.array([1., 1., 0., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #113 \t", len(np.nonzero(np.abs(predict - np.array([1., 0., 1., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #114 \t", len(np.nonzero(np.abs(predict - np.array([1., 0., 0., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #123 \t", len(np.nonzero(np.abs(predict - np.array([0., 1., 1., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #124 \t", len(np.nonzero(np.abs(predict - np.array([0., 1., 0., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #134 \t", len(np.nonzero(np.abs(predict - np.array([0., 0., 1., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #1123 \t", len(np.nonzero(np.abs(predict - np.array([1., 1., 1., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #1124 \t", len(np.nonzero(np.abs(predict - np.array([1., 1., 0., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #1134 \t", len(np.nonzero(np.abs(predict - np.array([1., 0., 1., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #1234 \t", len(np.nonzero(np.abs(predict - np.array([0., 1., 1., 1.])).sum(axis=1) < 0.1)[0]))

Prideicted Only class #1
Class #11 	 1
Class #12 	 0
Class #13 	 0
Class #14 	 44
Class #112 	 0
Class #113 	 0
Class #114 	 0
Class #123 	 0
Class #124 	 17
Class #134 	 0
Class #1123 	 0
Class #1124 	 0
Class #1134 	 0
Class #1234 	 0


In [ ]:
print("True")
print("Only class #1")
print("Invalid \t", len(np.nonzero(np.abs(y3_test - np.array([0., 0., 0., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #11 \t", len(np.nonzero(np.abs(y3_test - np.array([1., 0., 0., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #12 \t", len(np.nonzero(np.abs(y3_test - np.array([0., 1., 0., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #13 \t", len(np.nonzero(np.abs(y3_test - np.array([0., 0., 1., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #14 \t", len(np.nonzero(np.abs(y3_test - np.array([0., 0., 0., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #112 \t", len(np.nonzero(np.abs(y3_test - np.array([1., 1., 0., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #113 \t", len(np.nonzero(np.abs(y3_test - np.array([1., 0., 1., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #114 \t", len(np.nonzero(np.abs(y3_test - np.array([1., 0., 0., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #123 \t", len(np.nonzero(np.abs(y3_test - np.array([0., 1., 1., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #124 \t", len(np.nonzero(np.abs(y3_test - np.array([0., 1., 0., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #134 \t", len(np.nonzero(np.abs(y3_test - np.array([0., 0., 1., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #1123 \t", len(np.nonzero(np.abs(y3_test - np.array([1., 1., 1., 0.])).sum(axis=1) < 0.1)[0]))
print("Class #1124 \t", len(np.nonzero(np.abs(y3_test - np.array([1., 1., 0., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #1134 \t", len(np.nonzero(np.abs(y3_test - np.array([1., 0., 1., 1.])).sum(axis=1) < 0.1)[0]))
print("Class #1234 \t", len(np.nonzero(np.abs(y3_test - np.array([0., 1., 1., 1.])).sum(axis=1) < 0.1)[0]))

True
Only class #1
Invalid 	 5
Class #11 	 3
Class #12 	 22
Class #13 	 0
Class #14 	 29
Class #112 	 0
Class #113 	 0
Class #114 	 1
Class #123 	 0
Class #124 	 1
Class #134 	 1
Class #1123 	 0
Class #1124 	 0
Class #1134 	 0
Class #1234 	 0
